In [4]:
import os,sys,inspect
import json
import numpy as np
import renderer


currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 


import util




# just for plotting
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [5]:


# openexr

def translation_matrix(direction):
    # Return matrix to translate by direction vector.
    M = np.identity(4)
    M[:3, 3] = direction[:3]
    return M

def rotation_matrix(angle, direction):
    # Return matrix to rotate about axis defined by point and direction.
    sina = np.sin(angle)
    cosa = np.cos(angle)
    #direction = unit_vector(direction[:3])
    # rotation matrix around unit vector
    R = np.diag([cosa, cosa, cosa])
    R += np.outer(direction, direction) * (1.0 - cosa)
    direction *= sina
    R += np.array([[ 0.0,         -direction[2],  direction[1]],
                      [ direction[2], 0.0,          -direction[0]],
                      [-direction[1], direction[0],  0.0]])
    M = np.identity(4)
    M[:3, :3] = R
    return M

def load_camera( filename, id ):
    scn_file = filename
    json_file = open(scn_file)
    dd = json.load(json_file)
    json_file.close()
    cam = dd["cameras"][id]
    resx = cam["resx"]
    resy = cam["resx"]

    # http://www.sidefx.com/docs/houdini/ref/cameralenses
    pixel_aspect = 1.0
    focal_length = cam["camera.fl"]
    aperture_x = cam["camera.horizontalFilmAperture"]
    aperture_y = (resy*aperture_x) / (resx*pixel_aspect)
    hfovy_deg = np.rad2deg(2.0*np.arctan( (aperture_y/2.0) / focal_length ))

    # assemble localtransform
    translate = translation_matrix( np.array([cam["transform.tx"], cam["transform.ty"], cam["transform.tz"]]) )
    rotate_x = rotation_matrix( np.deg2rad(cam["transform.rx"]), np.array([1.0, 0.0, 0.0]) )
    rotate_y = rotation_matrix( np.deg2rad(cam["transform.ry"]), np.array([0.0, 1.0, 0.0]) )
    rotate_z = rotation_matrix( np.deg2rad(cam["transform.rz"]), np.array([0.0, 0.0, 1.0]) )

    cam2world = translate.dot(rotate_z.dot(rotate_y.dot(rotate_x)))

    camera = renderer.create_perspective_camera(resx, resy, hfovy_deg)
    camera.setCameraToWorldTransform(cam2world)

    return camera
